# LightAutoML Baselines - Rossmann Store Sales

В этом ноутбуке мы рассмотрим создание нескольких бейзлайнов с использованием LightAutoML (LAMA) для задачи регрессии (прогнозирование продаж).

**Особенности реализации:**
- Используем данные в "первозданном виде" (без ручной генерации признаков).
- Не делаем явный сплит (используем кросс-валидацию).
- Учитываем временную структуру данных (TimeSeriesSplit).
- Проверяем целостность данных (Store IDs).

In [51]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import logging
import sys
import matplotlib.pyplot as plt

# Logging setup
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
import os
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# LightAutoML imports
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report import ReportDeco
from lightautoml.transformers.base import LAMLTransformer, SequentialTransformer, UnionTransformer, ColumnsSelector
from lightautoml.transformers.numeric import QuantileTransformer, StandardScaler
from lightautoml.transformers.categorical import LabelEncoder
from lightautoml.dataset.roles import NumericRole, CategoryRole
from lightautoml.dataset.np_pd_dataset import PandasDataset
from lightautoml.dataset.roles import DatetimeRole, CategoryRole, NumericRole
from lightautoml.addons.autots.base import AutoTS

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

RANDOM_STATE = 42

## 1. Подготовка данных
Загружаем данные, проверяем соответствие магазинов и объединяем таблицы.

In [2]:
def load_and_process_data(data_dir='data', val_size=0.05):
    train = pd.read_csv(f'{data_dir}/train.csv', dtype={'StateHoliday': str})
    test = pd.read_csv(f'{data_dir}/test.csv', dtype={'StateHoliday': str})
    store = pd.read_csv(f'{data_dir}/store.csv')
    
    train_merged = train.merge(store, on='Store', how='left')
    test_merged = test.merge(store, on='Store', how='left')
    
    train_merged['Date'] = pd.to_datetime(train_merged['Date'])
    test_merged['Date'] = pd.to_datetime(test_merged['Date'])
    
    # Фильтр: только открытые магазины
    train_merged = train_merged[train_merged['Open'] == 1].sort_values('Date').reset_index(drop=True)
    
    # Сохрани маску закрытых в test
    test_closed_mask = test_merged['Open'] == 0
    
    # Дроп колонок
    drop_cols = ['Open', 'DayOfWeek', 'Customers']
    train_merged = train_merged.drop(columns=drop_cols)
    test_merged = test_merged.drop(columns=[c for c in drop_cols if c in test_merged.columns])
    
    # Разделение по дате
    split_date = train_merged['Date'].iloc[int(len(train_merged) * (1 - val_size))]
    train_final = train_merged[train_merged['Date'] < split_date].reset_index(drop=True)
    val_final = train_merged[train_merged['Date'] >= split_date].reset_index(drop=True)

    # Проверим наличие всех магазинов в train и val
    stores_train = set(train_final['Store'].unique())
    stores_val = set(val_final['Store'].unique())
    print(f"Магазинов только в train: {len(stores_train - stores_val)}")
    print(f"Магазинов только в val: {len(stores_val - stores_train)}")
    
    return train_merged, train_final, val_final, test_merged, test_closed_mask

train_full, train, val, test, test_closed_mask = load_and_process_data()
print(f"Train full: {train_full.shape}, Train: {train.shape}, Val: {val.shape}, Test: {test.shape}")

Магазинов только в train: 0
Магазинов только в val: 0
Train full: (844392, 15), Train: (801882, 15), Val: (42510, 15), Test: (41088, 15)


Из eda мы выяснили что при Open=0 продаж нет, что логично. Теперь оставим только открытые магазины в датасете и удалим признак Open. \
Для теста мы не можем просто удалить строки где Open=0, так что создадим маску для открытых магазинов и также признак Open. \
Интересный факт: у нас в train есть 54 записи, где нулевые продажи и Open=1. \
Также видим, что множества магазинов из test и train полностью совпадают!

## Настройка бейзлайна LAMA

In [3]:
TARGET_COLUMN = "Sales"
DATE_COLUMN = "Date"
ID_COLUMN = "Store"

univariate_roles = {
    "target": TARGET_COLUMN,
    DatetimeRole(seasonality=('d', 'm', 'wd'), base_date=True): DATE_COLUMN,
}

global_modelling_roles = {
    'target': TARGET_COLUMN,
    DatetimeRole(seasonality=['d', 'm', 'wd'], base_date=True): DATE_COLUMN,
    'id': ID_COLUMN,
    NumericRole(dtype=int): ['Promo', 'SchoolHoliday', 'Promo2'],
    CategoryRole(dtype=str): ['StateHoliday', 'StoreType', 'Assortment', 'PromoInterval'],
    NumericRole(dtype=float): ['CompetitionDistance', 'CompetitionOpenSinceMonth', 
                                'CompetitionOpenSinceYear', 'Promo2SinceWeek', 
                                'Promo2SinceYear']
}

task = Task(
    "multi:reg",
    greater_is_better=False,
    metric="mae",
    loss="mae"
)

seq_params = {
    "seq0": {
        "case": "next_values",                  
        "params": {
            "n_target": 1,                
            "history": 1,                            
            "step": 1, 
            "from_last": True,
            "test_last": True
        }
    }
}

transformers_params = {
    "lag_features": 1,
    "lag_time_features": 1,
    "diff_features": [1,],
}

automl = AutoTS(
    task,
    reader_params = {
        "seq_params": seq_params,
    },
    time_series_trend_params={
        "trend": False,
    },
    #time_series_pipeline_params=transformers_params
)

# # Time Series Split for Validation
# n_folds = 5
# cv = TimeSeriesSplit(n_splits=n_folds)
# cv_iter = list(cv.split(train))

multi:reg isn`t supported in lgb
multi:reg isn`t supported in xgb


In [4]:
train["id_2"] = train[ID_COLUMN]
oof_pred_seq = automl.fit_predict(train, roles=global_modelling_roles, verbose=4)
seq_test = automl.predict(train, return_raw=True)

[00:07:48] Stdout logging level is DEBUG.
INFO:lightautoml.automl.presets.base:Stdout logging level is DEBUG.
[00:07:48] Task: multi:reg

INFO:lightautoml.automl.presets.base:Task: multi:reg

[00:07:48] Start automl preset with listed constraints:
INFO:lightautoml.automl.presets.base:Start automl preset with listed constraints:
[00:07:48] - time: 3600.00 seconds
INFO:lightautoml.automl.presets.base:- time: 3600.00 seconds
[00:07:48] - CPU: 4 cores
INFO:lightautoml.automl.presets.base:- CPU: 4 cores
[00:07:48] - memory: 16 GB

INFO:lightautoml.automl.presets.base:- memory: 16 GB

[00:07:48] Layer 1 train process start. Time left 3599.71 secs
INFO:lightautoml.automl.base:Layer 1 train process start. Time left 3599.71 secs
[00:08:03] Start fitting Lvl_0_Pipe_0_Mod_0_RFSklearn ...
INFO:lightautoml.ml_algo.base:Start fitting Lvl_0_Pipe_0_Mod_0_RFSklearn ...
[00:08:03] Training params: {'bootstrap': True, 'ccp_alpha': 0.0, 'max_depth': None, 'max_features': 0.75, 'max_leaf_nodes': None, 'max

/Users/andrejustinov/Desktop/AutoML/.venv/lib/python3.12/site-packages/sklearn/base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[00:09:38] Score for RF model: -1399.940003
INFO2:lightautoml.ml_algo.random_forest:Score for RF model: -1399.940003
[00:09:38] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_RFSklearn =====
INFO2:lightautoml.ml_algo.base:===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_RFSklearn =====


/Users/andrejustinov/Desktop/AutoML/.venv/lib/python3.12/site-packages/sklearn/base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[00:11:20] Score for RF model: -1418.478108
INFO2:lightautoml.ml_algo.random_forest:Score for RF model: -1418.478108
[00:11:20] Fitting Lvl_0_Pipe_0_Mod_0_RFSklearn finished. score = -1409.2090555009702
INFO:lightautoml.ml_algo.base:Fitting Lvl_0_Pipe_0_Mod_0_RFSklearn finished. score = -1409.2090555009702
[00:11:20] Lvl_0_Pipe_0_Mod_0_RFSklearn fitting and predicting completed
INFO:lightautoml.ml_algo.base:Lvl_0_Pipe_0_Mod_0_RFSklearn fitting and predicting completed
[00:11:20] Time left 3387.41 secs

INFO:lightautoml.automl.base:Time left 3387.41 secs

[00:11:35] Start fitting Lvl_0_Pipe_1_Mod_0_LinearL2 ...
INFO:lightautoml.ml_algo.base:Start fitting Lvl_0_Pipe_1_Mod_0_LinearL2 ...
[00:11:35] Training params: {'tol': 1e-06, 'max_iter': 100, 'cs': [1e-05, 5e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000], 'early_stopping': 2, 'categorical_idx': [], 'embed_sizes': (), 'data_size': 23}
DEBUG:lightautoml.ml_algo.base:Tra

In [6]:
train.isna().sum()

Store                             0
Date                              0
Sales                             0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2072
CompetitionOpenSinceMonth    255115
CompetitionOpenSinceYear     255115
Promo2                            0
Promo2SinceWeek              402521
Promo2SinceYear              402521
PromoInterval                402521
id_2                              0
dtype: int64

In [ ]:
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.reader.base import PandasToPandasReader

reader = PandasToPandasReader(task=task, cv=n_folds, random_state=42)
train_dataset = reader.fit_read(train_data, roles=roles, cv_iter=cv_iter)

# features_pipeline применяется на каждом фолде отдельно
ml_pipe = MLPipeline(
    [BoostLGBM()], 
    pre_selection=None,
    features_pipeline=custom_transform  # Трансформер, не данные!
)

# Внутри fit_predict:
# для каждого фолда:
#   custom_transform.fit(train_fold)
#   train_transformed = custom_transform.transform(train_fold)
#   val_transformed = custom_transform.transform(val_fold)
#   model.fit(train_transformed)
#   pred = model.predict(val_transformed)
oof_pred = ml_pipe.fit_predict(train_dataset)

## LAMA Baseline

In [ ]:
# Numeric transformers
from lightautoml.transformers.numeric import (
    NaNFlags,
    FillnaMedian, 
    FillnaMean,
    FillInf,
    LogOdds,
    StandardScaler,
    QuantileBinning,
    QuantileTransformer
)

# Base transformers
from lightautoml.transformers.base import (
    LAMLTransformer,
    SequentialTransformer,
    UnionTransformer,
    ColumnsSelector,
    ColumnwiseUnion,
    ChangeRoles,
    ConvertDataset
)

# Decomposition
from lightautoml.transformers.decomposition import (
    PCATransformer,
    SVDTransformer
)

# GroupBy
from lightautoml.transformers.groupby import GroupByTransformer

# Sequential (для временных рядов)
from lightautoml.transformers.seq import (
    SeqLagTransformer,
    SeqDiffTransformer,
    SeqNumCountsTransformer,
    SeqStatisticsTransformer
)

from lightautoml.dataset.roles import (
    NumericRole,
    CategoryRole,
    DatetimeRole,
    DropRole,
    FoldsRole,
    GroupRole,
    TargetRole,
    WeightsRole,
    TreatmentRole,
    IdRole
)

from lightautoml.automl.base import AutoML
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.linear_sklearn import LinearLBFGS
from lightautoml.ml_algo.boost_cb import BoostCB
from lightautoml.ml_algo.tuning.optuna import OptunaTuner

# Бустинг
boosting_pipe = SequentialTransformer([
    UnionTransformer([
        SequentialTransformer([
            ColumnsSelector(['CompetitionDistance', 'CompetitionOpenSinceMonth', 
                           'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']),
            NaNFlags(),
            FillnaMedian()
        ]),
        SequentialTransformer([
            ColumnsSelector(['Store', 'Sales', 'CompetitionDistance']),
            GroupByTransformer(
                group_col='Store',
                numeric_cols=['Sales', 'CompetitionDistance'],
                categorical_cols=[],
                used_transforms=['delta_mean', 'delta_median', 'std']
            )
        ])
    ])
])

In [61]:
train.to_csv('example_train.csv')

[02:15:17] Train data shape: (801882, 16)

INFO:lightautoml.reader.base:Train data shape: (801882, 16)



TypeError: 'CategoryRole' object is not iterable

In [ ]:
train_dataset = PandasDataset(data=train, roles=roles)
boosting_pipe.fit_transform(train_dataset)

/Users/andrejustinov/Desktop/AutoML/.venv/lib/python3.12/site-packages/lightautoml/transformers/groupby.py:185: RuntimeWarning: Mean of empty slice
  return np.nanmean(vals)
/Users/andrejustinov/Desktop/AutoML/.venv/lib/python3.12/site-packages/lightautoml/transformers/groupby.py:181: RuntimeWarning: All-NaN slice encountered
  return np.nanmedian(vals)
/Users/andrejustinov/Desktop/AutoML/.venv/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
automl_fast = TabularAutoML(
    task=task,
    timeout=10000,
    cpu_limit=1,
    reader_params={'cv': n_folds, 'random_state': 42}
)

automl_reported = rd(automl_fast)

# Передаем cv_iter=cv. LAMA будет использовать этот итератор для разбиения.
# Так как данные отсортированы по дате, это обеспечит корректную валидацию на будущем.
oof_pred_fast = automl_reported.fit_predict(train_data, roles=roles, cv_iter=cv_iter, verbose=1)

[14:28:34] Stdout logging level is INFO.
[14:28:34] Task: reg

[14:28:34] Start automl preset with listed constraints:
[14:28:34] - time: 10000.00 seconds
[14:28:34] - CPU: 1 cores
[14:28:34] - memory: 16 GB

[14:28:34] Train data shape: (674844, 18)

[14:28:35] Layer 1 train process start. Time left 9999.06 secs
[14:28:42] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[14:29:03] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -2370262.0
[14:29:03] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[14:29:03] Time left 9971.45 secs

[14:29:21] Selector_LightGBM fitting and predicting completed
[14:29:29] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[14:30:28] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = -1945691.5
[14:30:28] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[14:30:28] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ... Time budget is 300.00 secs


Optimization Progress:   0%|          | 0/101 [00:00<?, ?it/s]

INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-21912d5c-4fb6-4ae0-9e7c-6bf03844e84e
INFO:optuna.study.study:Trial 0 finished with value: -1758666.0 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 244, 'bagging_fraction': 0.8659969709057025, 'min_sum_hessian_in_leaf': 0.24810409748678125, 'reg_alpha': 2.5361081166471375e-07, 'reg_lambda': 2.5348407664333426e-07}. Best is trial 0 with value: -1758666.0.


Optimization Progress:   1%|          | 1/101 [00:02<04:54,  2.94s/it, best_trial=0, best_value=-1.76e+6]

INFO:optuna.study.study:Trial 1 finished with value: -1746659.875 and parameters: {'feature_fraction': 0.5290418060840998, 'num_leaves': 223, 'bagging_fraction': 0.8005575058716043, 'min_sum_hessian_in_leaf': 0.679657809075816, 'reg_alpha': 1.5320059381854043e-08, 'reg_lambda': 5.360294728728285}. Best is trial 1 with value: -1746659.875.


...

Optimization Progress:  99%|█████████▉| 100/101 [04:41<00:02,  2.60s/it, best_trial=26, best_value=-1.87e+6]

INFO:optuna.study.study:Trial 100 finished with value: -1871831.375 and parameters: {'max_depth': 4, 'nan_mode': 'Min', 'l2_leaf_reg': 7.111686218113018, 'min_data_in_leaf': 1}. Best is trial 26 with value: -1871800.0.


Optimization Progress: 100%|██████████| 101/101 [04:44<00:00,  2.82s/it, best_trial=26, best_value=-1.87e+6]

[14:43:52] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost completed
[14:43:52] Start fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ...


[14:46:09] Fitting Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost finished. score = -2190724.25
[14:46:09] Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost fitting and predicting completed
[14:46:09] Time left 8945.48 secs

[14:46:09] Layer 1 training completed.

[14:46:09] Blending: optimization starts with equal weights. Score = -1937210.7500000
[14:46:09] Blending: iteration 0: score = -1900068.3750000, weights = [0.         0.22523865 0.56189543 0.10297609 0.10988986]
[14:46:09] Blending: iteration 1: score = -1877564.2500000, weights = [0.13000274 0.05743735 0.7296039  0.         0.08295603]
[14:46:09] Blending: iteration 2: score = -1874447.1250000, weights = [0.1773768  0.         0.76612157 0.         0.05650163]
[14:46:10] Blending: iteration 3: score = -1874231.8750000, weights = [0.1911324  0.         0.75426215 0.         0.05460551]
[14:46:10] Blending: iteration 4: score = -1874152.3750000, weights = [0.22103201 0.         0.778968   0.         0.        ]
[14:46:10] Blending: best score = -1874152.

In [80]:
train_lama = automl_fast.reader.read(train_data)
val_lama = automl_fast.reader.read(val_data)
pipeline = automl_fast.get_feature_pipeline("gbm")

train_feats = pipeline.fit_transform(train_lama)
val_feats = pipeline.transform(val_lama)

val_processed_df = pd.DataFrame(val_feats.data, columns=val_feats.features)

In [86]:
val_processed_df

,freq__Assortment,freq__PromoInterval,le__DayOfWeek,le__Promo2SinceYear,inter__(Store__CompetitionDistance),inter__(Store__Promo2SinceWeek),inter__(Store__Promo2SinceYear),inter__(CompetitionDistance__Promo2SinceWeek),inter__(CompetitionDistance__Promo2SinceYear),inter__(Promo2SinceWeek__Promo2SinceYear),inter__(Store__CompetitionDistance__Promo2SinceWeek),inter__(Store__CompetitionDistance__Promo2SinceYear),inter__(Store__Promo2SinceWeek__Promo2SinceYear),inter__(CompetitionDistance__Promo2SinceWeek__Promo2SinceYear),CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo,Promo2,SchoolHoliday,ord__CompetitionDistance,ord__Promo2SinceWeek,ord__StateHoliday,ord__Store,ord__StoreType,season_y__Date,season_m__Date,season_wd__Date
0,313264.0,191627.0,3.0,5.0,790.0,790.0,790.0,598.0,551.0,32.0,790.0,790.0,790.0,607.0,8.0,2010.0,1.0,1.0,0.0,4880.0,18.0,NaN,372.0,NaN,2015.0,1.0,4.0
1,313264.0,78384.0,3.0,3.0,489.0,489.0,489.0,381.0,369.0,48.0,489.0,489.0,489.0,401.0,4.0,2005.0,1.0,1.0,0.0,1450.0,45.0,NaN,368.0,NaN,2015.0,1.0,4.0
2,313264.0,340259.0,3.0,8.0,516.0,516.0,516.0,401.0,398.0,56.0,516.0,516.0,516.0,422.0,4.0,2014.0,1.0,0.0,0.0,5870.0,NaN,NaN,369.0,NaN,2015.0,1.0,4.0
3,355018.0,191627.0,3.0,3.0,931.0,931.0,931.0,730.0,676.0,34.0,931.0,931.0,931.0,725.0,10.0,2000.0,1.0,1.0,0.0,8250.0,31.0,NaN,370.0,NaN,2015.0,1.0,4.0
4,313264.0,78384.0,3.0,5.0,524.0,524.0,524.0,406.0,379.0,22.0,524.0,524.0,524.0,425.0,7.0,2009.0,1.0,1.0,0.0,1970.0,45.0,NaN,371.0,NaN,2015.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169489,355018.0,191627.0,3.0,3.0,848.0,848.0,848.0,651.0,602.0,52.0,848.0,848.0,848.0,643.0,11.0,2013.0,1.0,1.0,1.0,17650.0,37.0,NaN,745.0,NaN,2015.0,7.0,4.0
169490,313264.0,64574.0,3.0,7.0,558.0,558.0,558.0,451.0,422.0,27.0,558.0,558.0,558.0,446.0,2.0,2011.0,1.0,1.0,1.0,4330.0,35.0,NaN,746.0,NaN,2015.0,7.0,4.0
169491,313264.0,340259.0,3.0,8.0,1000.0,1000.0,1000.0,764.0,721.0,56.0,1000.0,1000.0,1000.0,771.0,8.0,2008.0,1.0,0.0,1.0,45740.0,NaN,NaN,747.0,NaN,2015.0,7.0,4.0
169492,313264.0,340259.0,3.0,8.0,325.0,325.0,325.0,261.0,243.0,56.0,325.0,325.0,325.0,266.0,NaN,NaN,1.0,0.0,1.0,11900.0,NaN,NaN,741.0,NaN,2015.0,7.0,4.0


In [89]:
val_processed_df.isnull().sum()

freq__Assortment                                                       0
freq__PromoInterval                                                    0
le__DayOfWeek                                                          0
le__Promo2SinceYear                                                    0
inter__(Store__CompetitionDistance)                                    0
inter__(Store__Promo2SinceWeek)                                        0
inter__(Store__Promo2SinceYear)                                        0
inter__(CompetitionDistance__Promo2SinceWeek)                          0
inter__(CompetitionDistance__Promo2SinceYear)                          0
inter__(Promo2SinceWeek__Promo2SinceYear)                              0
inter__(Store__CompetitionDistance__Promo2SinceWeek)                   0
inter__(Store__CompetitionDistance__Promo2SinceYear)                   0
inter__(Store__Promo2SinceWeek__Promo2SinceYear)                       0
inter__(CompetitionDistance__Promo2SinceWeek__Promo

In [88]:
automl_fast.plot_pdp(val_processed_df, feature_name="ord__CompetitionDistance")

AssertionError: 

In [62]:
automl_fast.plot_pdp(val_data, feature_name="CompetitionDistance")

100%|██████████| 10/10 [01:58<00:00, 11.84s/it]


KeyError: nan

## LAMA baseline, custom transforms

In [48]:
train_data

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1097,2,2013-01-01,5961,1405,1,0,a,1,b,b,720.0,3.0,2002.0,0,NaN,NaN,NaN
1,85,2,2013-01-01,4220,619,1,0,a,1,b,a,1870.0,10.0,2011.0,0,NaN,NaN,NaN
2,259,2,2013-01-01,6851,1444,1,0,a,1,b,b,210.0,NaN,NaN,0,NaN,NaN,NaN
3,262,2,2013-01-01,17267,2875,1,0,a,1,b,a,1180.0,5.0,2013.0,0,NaN,NaN,NaN
4,274,2,2013-01-01,3102,729,1,0,a,1,b,b,3640.0,NaN,NaN,1,10.0,2013.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844333,745,5,2015-07-31,8363,746,1,1,0,1,a,a,17650.0,11.0,2013.0,1,37.0,2009.0,"Jan,Apr,Jul,Oct"
844334,746,5,2015-07-31,9082,638,1,1,0,1,d,c,4330.0,2.0,2011.0,1,35.0,2011.0,"Mar,Jun,Sept,Dec"
844335,747,5,2015-07-31,10708,826,1,1,0,1,c,c,45740.0,8.0,2008.0,0,NaN,NaN,NaN
844336,741,5,2015-07-31,11253,1137,1,1,0,1,d,c,11900.0,NaN,NaN,0,NaN,NaN,NaN


In [ ]:
automl_custom = TabularAutoML(
    task=task,
    timeout=1200,
    cpu_limit=4,
    reader_params={'cv': n_folds, 'random_state': 42},
    general_params={'use_algos': [['lgb', 'cb']]}
)

oof_pred_custom = automl_custom.fit_predict(train_data, roles=roles, cv_iter=cv)
print(f"Custom Baseline OOF RMSPE: {rmspe(train_data['Sales'].values, oof_pred_custom.data[:, 0]):.4f}")